In [1]:
import pandas as pd
import numpy as np
from gravity_utils import *

In [2]:
# constants
filtered_columns = ['Id' ,'bostad_komkod', 'bostad_SAMS', 'weight', 'rf1_komkod',
                    'Id_time', 'rf4_komkod', 'rf4_Samskod']


# Upsample based on idividvikt from RVU

In [ ]:
resfil_raw = pd.read_csv('data/rvu/RVU_resfil.csv')
resfil_raw = resfil_raw.loc[:, filtered_columns + ['individvikt']]
#df_resfil = df_resfil[df_resfil['ärende_2']==1]
upsampled_resfil = draw_population(resfil_raw, resfil_raw.individvikt.astype(int))

/home/ai6644/.local/lib/python3.5/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [8]:
prod_all_trips = upsampled_resfil.loc[(upsampled_resfil.rf1_komkod >= 1200) & (upsampled_resfil.rf1_komkod < 1300)]
prod_all_trips = prod_all_trips.groupby(['rf1_komkod'])['Id_time'].count().rename('work_trips')

# Production linear regression

In [9]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn import metrics
import statsmodels.api as sm
from scipy import stats
import stat

In [10]:
a2 = read_shapefile('data/GIS/A2_samsSW_2012_shp/A2_sw_region.shp')
a2['KOMMUN'] = a2.KOMMUN.astype(int)
a2['SAMSCODE'] = a2.SAMSCODE.astype(int)
night_pop = a2.groupby(['KOMMUN']).sum()
night_pop = night_pop[(night_pop.index > 1200) & (night_pop.index < 1300)]

b2 = read_shapefile('data/GIS/B2_sams_2013/B2_sams.shp')
b2['KOMMUN'] = b2.KOMMUN.astype(int)
b2['SAMSCODE'] = b2.SAMSCODE.astype(int)
sex_distr = b2.groupby(['KOMMUN']).sum()
sex_distr = sex_distr[(sex_distr.index > 1200) & (sex_distr.index < 1300)]

b1_buildings = pd.read_csv('data/GIS/B1_samsSW_20131231_shp/B1_sams_with_nbuildings.csv', sep=';', index_col=0)
prod_buildings = b1_buildings.groupby(['KOMMUN'])['nProduction', 'nAttraction', 'TotBef', 'small_building', 'appt_build', 'multi_appartment_building'].sum()

In [11]:
prod_all_x = pd.concat([night_pop, sex_distr, prod_buildings], axis=1)
prod_all_x.head()

,SAMSCODE,Offentliga,Naringsliv,Totalt,SAMSCODE,Man,Kv,TotKon,nProduction,nAttraction,TotBef,small_building,appt_build,multi_appartment_building
KOMMUN,,,,,,,,,,,,,,
1214,206380153,1797,4469,6266,206380153,6889,6427,13316,5111,693,13317,4835,136,139
1230,123000075,2964,7828,10792,123000075,11193,11447,22640,6880,510,22640,6509,281,90
1231,86170035,2018,5170,7188,86170035,8558,8505,17063,3262,488,17063,3017,186,59
1233,308250325,3194,12424,15618,308250325,16679,17107,33786,16269,1103,33786,14571,321,125
1256,150720078,1597,4498,6095,150720078,6939,6724,13663,5792,2880,13663,5520,202,68


### testing a model

In [12]:
x = prod_all_x.drop(axis=1, labels=['Man', 'Naringsliv', 'TotKon', 'nAttraction', 'Kv', 'Totalt', 'multi_appartment_building', 'nProduction', 'small_building', 'appt_build'])
#x = all_x.loc[:, ['TotBef', 'Naringsliv', ]]
y = pd.DataFrame(prod_all_trips)

est = sm.OLS(y, x)
est2 = est.fit()
print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:             work_trips   R-squared:                       0.989
Model:                            OLS   Adj. R-squared:                  0.988
Method:                 Least Squares   F-statistic:                     892.5
Date:                Tue, 03 Sep 2019   Prob (F-statistic):           2.07e-29
Time:                        15:03:16   Log-Likelihood:                -358.37
No. Observations:                  33   AIC:                             722.7
Df Residuals:                      30   BIC:                             727.2
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
SAMSCODE     -3.8e-06   3.49e-06     -1.089      0.2

### actual model

In [13]:
x = prod_all_x.loc[:, ['Offentliga', 'TotBef']]
y = pd.DataFrame(prod_all_trips)

production_model = LinearRegression(fit_intercept=0)
scores = []

production_model.fit(x, y)
score = production_model.score(x, y)
print('Scores')
print(scores, scores)
print('Model')
print(production_model.intercept_, production_model.coef_)
pred_production = pd.DataFrame(production_model.predict(x))
pred_production['ind'] = x.index
pred_production.set_index('ind', inplace=True)

Scores
[] []
Model
0.0 [[4.95985625 1.13555962]]


# Attraction Linear regression

In [14]:
#att_work_trips = upsampled_resfil[upsampled_resfil['ärende_2'] == 1]

attr_all_trips = upsampled_resfil.loc[(upsampled_resfil.rf4_komkod >= 1200) & (upsampled_resfil.rf4_komkod < 1300)]
attr_all_trips = attr_all_trips.groupby(['rf4_komkod'])['Id_time'].count().rename('all_attr_trips')

In [15]:
a4 = read_shapefile('data/GIS/A4_samsSW_2012_shp/A4_sw_region.shp')
a4['KOMMUN'] = a4.KOMMUN.astype(int)
day_pop = a4.groupby(['KOMMUN']).sum()
day_pop = day_pop[(day_pop.index > 1200) & (day_pop.index < 1300)]

attr_buildings = b1_buildings.groupby(['KOMMUN'])['nProduction', 'nAttraction', 'TotBef', 'nIndustri', 'nEkonomi'].sum()

In [16]:
attr_x = pd.concat([day_pop, sex_distr, attr_buildings], axis=1)
attr_x.head()



,Offentliga,Naringsliv,Totalt,SAMSCODE,Man,Kv,TotKon,nProduction,nAttraction,TotBef,nIndustri,nEkonomi
KOMMUN,,,,,,,,,,,,
1214,1163.0,2152.0,3315.0,206380153,6889,6427,13316,5111,693,13317,142,130
1230,1121.0,4312.0,5433.0,123000075,11193,11447,22640,6880,510,22640,219,44
1231,1365.0,6401.0,7766.0,86170035,8558,8505,17063,3262,488,17063,262,47
1233,1128.0,6605.0,7733.0,308250325,16679,17107,33786,16269,1103,33786,262,271
1256,854.0,3074.0,3928.0,150720078,6939,6724,13663,5792,2880,13663,270,161


In [97]:
x = attr_x.drop(columns=['Man', 'Naringsliv', 'TotKon', 'Kv', 'nEkonomi', 'nIndustri', 'nProduction', 'nAttraction', 'SAMSCODE'])
y = pd.DataFrame(attr_all_trips)

est = sm.OLS(y, x)
est2 = est.fit()
print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:         all_attr_trips   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.998
Method:                 Least Squares   F-statistic:                     5180.
Date:                Tue, 03 Sep 2019   Prob (F-statistic):           8.38e-41
Time:                        15:53:03   Log-Likelihood:                -329.40
No. Observations:                  33   AIC:                             664.8
Df Residuals:                      30   BIC:                             669.3
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Offentliga     2.7119      0.584      4.648      0.0

In [98]:
x = attr_x.loc[:, ['Offentliga', 'Totalt', 'TotBef']]
y = (pd.DataFrame(attr_all_trips))

attraction_model = LinearRegression(fit_intercept=0)
scores = []

attraction_model.fit(x, y)
scores = attraction_model.score(x, y)
print('Scores')
print(scores, scores)
print('Model')
print(attraction_model.intercept_, attraction_model.coef_)
pred_attraction = pd.DataFrame(attraction_model.predict(x))
pred_attraction['ind'] = x.index
pred_attraction.set_index('ind', inplace=True)

Scores
0.997421267394188 0.997421267394188
Model
0.0 [[2.71192777 2.0514674  0.42853402]]


# Gravity Model for municipalities

In [123]:
kommun_distance_matrix = pd.read_csv('data/kommun_distance_matrix.csv', sep=';', index_col=[0])
#the thing above is differently sorted
kommun_distance_matrix = kommun_distance_matrix.sort_index().T.sort_index().T

In [124]:
kommun_distance_matrix.replace(0, np.nan, inplace=True)
for (idx, value) in kommun_distance_matrix.min().iteritems():
    kommun_distance_matrix.at[int(idx), idx] = value * (2.0 / 3)

In [125]:
f = kommun_distance_matrix.values
cost_matrix = np.power((f/10000),0.1) * np.exp(-0.5*(f/10000))
Trips1 = fratar_double_constrained(prodA = pred_production.values.flatten(),
                                   attrA = pred_attraction.values.flatten(),
                                   cost_matrix = cost_matrix)

Checking production, attraction balancing:
Production:  2246455.1728156987
Attraction:  1987796.6671588297
Productions and attractions do not balance, attractions will be scaled to productions!


In [126]:
komm_trips = pd.DataFrame(Trips1, columns=kommun_distance_matrix.index, index=kommun_distance_matrix.index)
komm_trips.head()

,1214,1230,1231,1233,1256,1257,1260,1261,1262,1263,...,1282,1283,1284,1285,1286,1287,1290,1291,1292,1293
1214,1208.531634,123.714331,150.113105,99.483368,40.941624,154.757178,773.866316,1400.265510,374.123575,78.731155,...,2339.481909,4333.626546,441.571854,1799.657207,100.520376,115.359401,217.924983,23.340534,996.344740,651.273634
1230,121.026119,1333.458092,1047.019283,915.389514,14.400095,16.221792,56.761187,587.589363,761.975391,711.124279,...,764.856277,859.063717,86.738897,1120.909707,626.592923,1074.537199,221.746969,148.100069,132.391771,204.109176
1231,97.824180,699.684263,844.517336,714.551363,6.797411,13.168549,45.972518,471.443166,604.521697,491.482597,...,615.740672,695.275155,70.283551,534.725163,192.038297,841.696693,104.742035,59.350338,107.359485,96.455546
1233,93.410563,934.669268,1141.649987,4951.826467,7.382623,12.374958,43.579822,461.019835,609.337709,1161.319738,...,595.920011,660.841481,66.515700,600.073577,443.909068,5667.509478,114.044114,73.371886,101.304359,105.202230
1256,52.237602,19.586753,15.233017,9.896703,1892.242419,217.409287,48.885986,28.774083,20.643115,7.867700,...,30.118013,335.620883,33.761864,81.038833,69.637406,11.434659,11521.566084,207.338723,149.905277,4801.773604


In [127]:
compare_trips = upsampled_resfil.loc[:,['rf1_komkod', 'rf4_komkod']]
compare_trips['rf4_komkod'] = compare_trips.rf4_komkod.astype(int)
rvu_trips = compare_trips[compare_trips.rf1_komkod == 1280].groupby(['rf4_komkod']).count()
malmo_trips = komm_trips.loc[1280]
malmo_trips.set_axis(malmo_trips.reset_index()['index'].astype(int), inplace=True)
original_resfil = draw_population(resfil_raw, resfil_raw.individvikt.astype(int))
#original_resfil = original_resfil[original_resfil['ärende_2']==1]
compare_original_trips = original_resfil.loc[:,['rf1_komkod', 'rf4_komkod']]
compare_original_trips['rf4_komkod'] = compare_original_trips.rf4_komkod.astype(int)
original_rvu_trips = compare_original_trips[compare_original_trips.rf1_komkod == 1280].groupby(['rf4_komkod']).count()
res = pd.concat([malmo_trips.astype(int), rvu_trips, original_rvu_trips, original_rvu_trips.rf1_komkod / malmo_trips], axis=1)
res.columns = ['gravity', 'rvu', 'original_rvu','difference']
res

,gravity,rvu,original_rvu,difference
rf4_komkod,,,,
99,NaN,69313,69313,NaN
1214,1257.0,1153,1153,0.916619
1230,9449.0,4328,4328,0.458022
1231,14239.0,8800,8800,0.617988
1233,16515.0,9237,9237,0.559298
1256,100.0,231,231,2.291466
1257,168.0,183,183,1.086567
1260,589.0,305,305,0.517253
1261,6117.0,3623,3623,0.592229


In [128]:
np.square(res.gravity - res.original_rvu).mean()

134874573.0909091

In [129]:
res.difference.describe()

count    33.000000
mean      0.859815
std       0.563254
min       0.155473
25%       0.517253
50%       0.602312
75%       1.086567
max       2.392109
Name: difference, dtype: float64

In [130]:
res.gravity.sum() - res.gravity.loc[1280]

208309.0

In [131]:
res.gravity.loc[1233]

16515.0

In [132]:
res.gravity.loc[1287]

19534.0

In [133]:
komm_trips.loc[1265].sum()

32196.791099185175

In [134]:
attr_buildings.TotBef.loc[1265]

18386

In [135]:
print('sjöbo-west', komm_trips.loc[1265, [1280, 1231, 1230, 1281, 1262]].sum())
print('sjobo-south', komm_trips.loc[1265, [1286]].sum())
print('sjobo-east', komm_trips.loc[1265, [1291, 1270]].sum())
print('sjobo-north',komm_trips.loc[1265, [1266, 1267, 1293, 1276, 1275, 1257, 1273]].sum())

sjöbo-west 12716.230795587568
sjobo-south 6049.61409394332
sjobo-east 3403.9240039651595
sjobo-north 2662.652583999829


In [104]:
prodA = pred_production.values.flatten()
attrA = pred_attraction.values.flatten()
f = kommun_distance_matrix.values
c=-0.00001
cost_matrix = np.exp(c*f)
num_iter=10
num_iter_cal = 25
mtl = 17000
num_zones = len(prodA)
alpha = 0.5

trips = np.zeros((num_zones, num_zones))
print('Checking production, attraction balancing:')
sumP = sum(prodA)
sumA = sum(attrA)
print('Production: ', sumP)
print('Attraction: ', sumA)
if sumP != sumA:
    print('Productions and attractions do not balance, attractions will be scaled to productions!')
    attrA = attrA * (sumP / sumA)
    attrT = attrA.copy()
    prodT = prodA.copy()
else:
    print('Production, attraction balancing OK.')
    attrT = attrA.copy()
    prodT = prodA.copy()
    
trips = cost_matrix
#Run 2D balancing --->
computed_attractions = trips.sum(0)
computed_attractions[computed_attractions==0]=1
trips = trips * (attrT / computed_attractions)

computed_productions = trips.sum(1)
computed_productions[computed_productions==0]=1
trips = trips * np.reshape((prodT / computed_productions),[len(prodT),1])

for Iter in range(0, num_iter_cal):

    model_trip_len = (trips*f).sum() / trips.sum()
    c = c*(model_trip_len / mtl)**alpha
    cost_matrix = np.exp(c * f)
    for _ in range(0, num_iter):
        trips = cost_matrix
        #Run 2D balancing --->
        computed_attractions = trips.sum(0)
        computed_attractions[computed_attractions==0]=1
        trips = trips * (attrT / computed_attractions)

        computed_productions = trips.sum(1)
        computed_productions[computed_productions==0]=1
        trips = trips * np.reshape((prodT / computed_productions),[len(prodT),1])
#        for i in range(0, num_zones):
#            trips[i,:] = prodA[i]*attrA*cost_matrix[i,:]/max(0.000001, sum(attrA * cost_matrix[i,:]))
    print ('iteration: ', Iter, ' coefficient: ', c, ' average trip length (model): ', model_trip_len)

print ('target average trip length (observed): ', mtl) 
print ('final average trip length (model): ', model_trip_len)
print ('final logit scaling factor: ', c)

Checking production, attraction balancing:
Production:  2246455.1728156987
Attraction:  1987796.6671588297
Productions and attractions do not balance, attractions will be scaled to productions!
iteration:  0  coefficient:  -1.76324986275802e-05  average trip length (model):  52853.851334775005
iteration:  1  coefficient:  -2.8810137417604277e-05  average trip length (model):  45384.95022600421
iteration:  2  coefficient:  -4.173824139096725e-05  average trip length (model):  35680.140407964864
iteration:  3  coefficient:  -5.311361986074329e-05  average trip length (model):  27529.12643544784
iteration:  4  coefficient:  -6.144182516259602e-05  average trip length (model):  22749.15845843435
iteration:  5  coefficient:  -6.700951167406643e-05  average trip length (model):  20220.580120555176
iteration:  6  coefficient:  -7.056586446692369e-05  average trip length (model):  18852.34365006166
iteration:  7  coefficient:  -7.278143701689562e-05  average trip length (model):  18084.2641198

In [105]:
trips = pd.DataFrame(trips, columns=kommun_distance_matrix.index, index=kommun_distance_matrix.index)
trips.head()

,1214,1230,1231,1233,1256,1257,1260,1261,1262,1263,...,1282,1283,1284,1285,1286,1287,1290,1291,1292,1293
1214,3211.410560,60.076487,85.220678,36.207993,13.792569,58.587405,811.013851,1855.313102,324.741284,27.913070,...,2919.546239,4004.895506,317.791677,2784.640718,17.074695,26.764624,63.938760,2.371776,575.599380,426.954396
1230,31.698077,2300.333613,1265.306142,744.563859,1.663861,1.059529,7.781488,278.283874,640.864020,573.991014,...,295.984185,190.325355,15.102470,796.509605,185.314552,550.374926,40.324874,25.741283,14.870655,42.618487
1231,24.206310,696.819184,1241.532274,544.344205,0.547033,0.809112,5.942351,212.512122,489.397285,345.910104,...,226.029005,145.342396,11.533036,261.870897,30.633707,402.374354,13.257732,6.543917,11.356010,14.011810
1233,14.797371,637.628831,924.348364,12202.244191,0.419964,0.494612,3.632572,129.909134,299.169653,848.340773,...,138.172035,88.848131,7.050171,201.041640,75.128834,5453.169819,10.178131,6.106269,6.941954,10.757046
1256,5.521197,1.348916,0.953920,0.403014,3573.113197,39.906566,4.011718,1.573099,1.374436,0.310687,...,1.216499,28.552031,2.265627,8.105062,3.865764,0.297905,14017.050093,28.359606,11.772903,4114.126732


In [93]:
compare_trips = upsampled_resfil.loc[:,['rf1_komkod', 'rf4_komkod']]
compare_trips['rf4_komkod'] = compare_trips.rf4_komkod.astype(int)
rvu_trips = compare_trips[compare_trips.rf1_komkod == 1280].groupby(['rf4_komkod']).count()
malmo_trips = trips.loc[1280]
malmo_trips.set_axis(malmo_trips.reset_index()['index'].astype(int), inplace=True)
original_resfil = draw_population(resfil_raw, resfil_raw.individvikt.astype(int))
#original_resfil = original_resfil[original_resfil['ärende_2']==1]
compare_original_trips = original_resfil.loc[:,['rf1_komkod', 'rf4_komkod']]
compare_original_trips['rf4_komkod'] = compare_original_trips.rf4_komkod.astype(int)
original_rvu_trips = compare_original_trips[compare_original_trips.rf1_komkod == 1280].groupby(['rf4_komkod']).count()
res = pd.concat([malmo_trips.astype(int), rvu_trips, original_rvu_trips, original_rvu_trips.rf1_komkod / malmo_trips], axis=1)
res.columns = ['gravity', 'rvu', 'original_rvu','difference']
res

,gravity,rvu,original_rvu,difference
rf4_komkod,,,,
99,NaN,69313,69313,NaN
1214,204.0,1153,1153,5.649084
1230,5975.0,4328,4328,0.724273
1231,13213.0,8800,8800,0.666004
1233,11746.0,9237,9237,0.786353
1256,5.0,231,231,39.755895
1257,6.0,183,183,26.823772
1260,50.0,305,305,6.087218
1261,1791.0,3623,3623,2.021910


In [94]:
res.difference.describe()

count    33.000000
mean     10.434946
std      15.639436
min       0.666004
25%       1.715142
50%       3.759299
75%      15.004196
max      77.834863
Name: difference, dtype: float64

In [106]:
trips.loc[1265].sum()

32196.79109918518

In [121]:
print('sjöbo-west', trips.loc[1265, [1280, 1231, 1230, 1281, 1262]].sum())
print('sjobo-south', trips.loc[1265, [1286]].sum())
print('sjobo-east', trips.loc[1265, [1291, 1270]].sum())
print('sjobo-north',trips.loc[1265, [1266, 1267, 1293, 1276, 1275, 1257, 1273]].sum())

sjöbo-west 8952.250102566371
sjobo-south 7784.601128157108
sjobo-east 3259.7218499242867
sjobo-north 1977.0569242156362


In [84]:
resfil_raw = pd.read_csv('/home/ai6644/Malmo/Data/Travel_survey_Skane_2013/RVU_resfil.csv')
resfil_raw.head()

,Id,Id_time,förfl_nr,lillam,lillan,storaN,Stratum,mätdag,dagtyp,dagtyp_2,...,rf7_km,kl6_km,bostad_storst_småort_landsby,restyp,individvikt,individvikt_svarandemängd,urval_trompNr,rf1_TrompNr,rf4_TrompNr,filter_$
0,1000074.0,10000741.0,1.0,519.0,1608.0,7845.0,27.0,1.0,1.0,1.0,...,11.0,4.0,2.0,2.0,8.330817,0.201277,NaN,NaN,NaN,NaN
1,1000074.0,10000742.0,2.0,519.0,1608.0,7845.0,27.0,1.0,1.0,1.0,...,11.0,4.0,2.0,2.0,8.330817,0.201277,NaN,NaN,NaN,NaN
2,1000116.0,10001161.0,1.0,2118.0,6066.0,25563.0,22.0,1.0,1.0,1.0,...,8.0,3.0,3.0,2.0,22.281823,0.538341,NaN,NaN,NaN,NaN
3,1000116.0,10001162.0,2.0,2118.0,6066.0,25563.0,22.0,1.0,1.0,1.0,...,3.0,2.0,3.0,2.0,22.281823,0.538341,NaN,NaN,NaN,NaN
4,1000116.0,10001163.0,3.0,2118.0,6066.0,25563.0,22.0,1.0,1.0,1.0,...,3.0,2.0,3.0,2.0,22.281823,0.538341,NaN,NaN,NaN,NaN


In [90]:
(resfil_raw.rf7_km * resfil_raw.individvikt).sum() / (resfil_raw.individvikt.sum())

17.713863322894873